In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [28]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [4]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [7]:
# Parse our cleaned data.
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
emma_doc = nlp(emma)

In [10]:
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

In [11]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [17]:
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]


In [18]:
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

common_words = set(alicewords + persuasionwords)

In [19]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


In [22]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,lark,reflection,fashion,confusing,bind,sloop,stamp,mention,ladder,support,...,book,disgrace,unhappy,knave,slippery,convenient,eyelid,embarrassment,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


# Challenge 0:


### Gradient Boost

In [24]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split


#Gridsearchcv for GBR
params = {'n_estimators': 500,
          'max_depth': 7,
          'loss': 'deviance'}
# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9811912225705329

Test set score: 0.9027255639097744


It looks like the above dataset might have little overfitting.But test data has good accuracy more than 90%.

In [ ]:
from sklearn.model_selection import cross_val_score
print("Training data accuracy for GBR: ",cross_val_score(clf, X_train, y_train, cv=10))
print("Testing data accuracy for GBR: ",cross_val_score(clf, X_test, y_test, cv=10))

### SVC

In [ ]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(X_train, y_train)
from sklearn.model_selection import cross_val_score
print("Training data accuracy for GBR: ",cross_val_score(svm, X_train, y_train, cv=10))
print("Testing data accuracy for GBR: ",cross_val_score(svm, X_test, y_test, cv=10))

In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

# Challenge 1:

In [30]:

# Grab Whitman, clean and process
whitman = gutenberg.raw('whitman-leaves.txt')
whitman = re.sub(r'VOLUME \w+', '', whitman)
whitman = re.sub(r'CHAPTER \w+', '', whitman)
whitman = text_cleaner(whitman)
print(whitman[:100])

Come, said my soul, Such verses for my Body let us write, (for we are one,) That should I after retu


In [31]:
# Parse into doc using SpaCy
whitman_doc = nlp(whitman[0:25000])

In [ ]:
# Bag words
whitmanwords_punct = bag_of_words(whitman_doc)
alicewords_punct = bag_of_words(alice_doc)

# Combine Whitman and Alice bags
common_words = set(whitmanwords_punct + alicewords_punct)

# Group into sentences
whitman_sents = [[sent, 'Whitman'] for sent in whitman_doc.sents]

# Combine Whitman and Alice sentences
sentences = pd.DataFrame(whitman_sents + alice_sents)

# Get BoW Features
word_counts = bow_features(sentences, common_words)

Processing row 0


In [ ]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

In [ ]:
from sklearn.model_selection import cross_val_score
print('Scores:', cross_val_score(lr, X, Y, cv=5))


Setup and run model: Persuasion vs Whitman


The model is still good at identifying Austen's Persuasion vs. Whitman's Leaves, although there is quite a bit more variation between folds compared to Whitman vs. Alice.

In [ ]:

# Combine Whitman and Persuasion bags
common_words = set(whitmanwords_punct + persuasionwords_punct)

# Combine Whitman and Persuasion sentences
sentences = pd.DataFrame(whitman_sents + persuasion_sents)

# Get BoW Features
word_counts = bow_features(sentences, common_words)

In [ ]:

Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

In [ ]:
from sklearn.model_selection import cross_val_score
print('Scores:', cross_val_score(lr, X, Y, cv=5))